In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install kaggle

In [ ]:
from fastai.vision.widgets import *

In [ ]:
from fastai.vision.all import *


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivam17299/oral-cancer-lips-and-tongue-images")

print("Path to dataset files:", path)

In [ ]:
# Define the path to the dataset
data_path = Path("/kaggle/input/oral-cancer-lips-and-tongue-images/OralCancer")


data_path.ls()  # Check the folders inside

In [ ]:
dls = ImageDataLoaders.from_folder(
    data_path, 
    valid_pct=0.2,  # 20% images for validation
    item_tfms=Resize(224),  # Resize images to 224x224
    batch_tfms=aug_transforms()  # Apply data augmentation
)

In [ ]:
dls.show_batch(max_n=9)


In [ ]:
faces = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files, 
    splitter=RandomSplitter(seed=42),
    get_y=parent_label,  # Labels are folder names
    item_tfms=Resize(460),  # Step 1: Resize to a larger size
    batch_tfms=aug_transforms(size=224)  # Step 2: Apply augmentation & final resize
)

dls = faces.dataloaders("/kaggle/input/oral-cancer-lips-and-tongue-images/OralCancer")


In [ ]:
dls.show_batch(max_n=9)

In [ ]:
learn = vision_learner(dls, resnet18, metrics = error_rate )
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1, figsize=(17,4))

In [ ]:
learn.export('cancer_model.pkl')